In [1]:
import import_ipynb
import numpy as np
import GA
import pickle
import ANN
import matplotlib.pyplot as plt

importing Jupyter notebook from GA.ipynb
importing Jupyter notebook from ANN.ipynb


In [2]:
f = open("dataset_features.pkl", "rb")

In [3]:
data_inputs2 = pickle.load(f)

In [4]:
f.close()

In [5]:
features_STDs = np.std(a = data_inputs2, axis = 0)

In [6]:
data_inputs = data_inputs2[:, features_STDs > 50]
data_inputs

array([[1.576e+03, 3.380e+02, 6.160e+02, ..., 0.000e+00, 0.000e+00,
        1.070e+02],
       [1.769e+03, 8.630e+02, 1.117e+03, ..., 0.000e+00, 0.000e+00,
        3.700e+02],
       [1.847e+03, 8.600e+02, 1.115e+03, ..., 0.000e+00, 0.000e+00,
        3.780e+02],
       ...,
       [5.520e+02, 2.000e+00, 8.000e+00, ..., 5.000e+00, 1.000e+00,
        5.000e+00],
       [5.830e+02, 2.000e+00, 7.000e+00, ..., 0.000e+00, 0.000e+00,
        4.000e+00],
       [6.300e+02, 1.000e+00, 8.000e+00, ..., 5.000e+00, 0.000e+00,
        4.000e+00]])

In [7]:
f = open("outputs.pkl", "rb")
data_outputs = pickle.load(f)
f.close()

In [8]:
"""
Generate algorithm parameters:
    Mating Pool Size (Number of Parameters)
    Population Size
    Number of Generations
    Mutation Percent
"""

'\nGenerate algorithm parameters:\n    Mating Pool Size (Number of Parameters)\n    Population Size\n    Number of Generations\n    Mutation Percent\n'

In [9]:
sol_per_pop = 8
num_parents_mating = 4
num_generations = 1000
mutation_percent = 10

In [10]:
# Creating the initial population.
initial_pop_weights = []
for curr_sol in np.arange(0, sol_per_pop):
    
    HL1_neurons = 150
    input_HL1_weights = np.random.uniform(low = -0.1, high = 0.1, size = (data_inputs.shape[1], HL1_neurons))
    
    HL2_neurons = 60
    HL1_HL2_weights = np.random.uniform(low = -0.1, high = 0.1, size = (HL1_neurons, HL2_neurons))
    
    output_neurons = 4
    HL2_output_weights = np.random.uniform(low = -0.1, high = 0.1, size = (HL2_neurons, output_neurons))
    
    initial_pop_weights.append(np.array([input_HL1_weights, HL1_HL2_weights, HL2_output_weights])) 

In [11]:
pop_weights_mat = np.array(initial_pop_weights)
pop_weights_vector = GA.mat_to_vector(pop_weights_mat)

In [12]:
best_outputs = []
accuracies = np.empty(shape = (num_generations))

In [13]:
for generation in range(num_generations):
    print("Generation: ", generation)
    
    # Converting the solution from being vectors to matrices.
    pop_weights_mat = GA.vector_to_mat(pop_weights_vector, pop_weights_mat)
    
    # Measuring the fitness of each chromosomes in the population.
    fitness = ANN.fitness(pop_weights_mat, data_inputs, data_outputs, activation = "sigmoid")
    accuracies[generation] = fitness[0]
    print("Fitness")
    print(fitness)
    
    # Selecting the best parents in the population for mating.
    parents = GA.select_mating_pool(pop_weights_vector, fitness.copy(), num_parents_mating)
    print("Parents")
    print(fitness)
    
    # Generating next generation using crossover.
    offspring_crossover = GA.crossover(parents, 
                                       offspring_size = (pop_weights_vector.shape[0] - parents.shape[0], 
                                                         pop_weights_vector.shape[1]))
    print("Crossover")
    print(offspring_crossover)
    
    # Adding some variations to the offspring using mutation.
    offspring_mutation = GA.mutation(offspring_crossover, mutation_percent = mutation_percent)
    print("Mutation")
    print(offspring_mutation)
    
    # Creating the new population based on the parents and offspring.
    pop_weights_vector[0: parents.shape[0], :] = parents
    pop_weights_vector[parents.shape[0]:, :] = offspring_mutation
    
pop_weights_mat = GA.vector_to_mat(pop_weights_vector, pop_weights_mat)
best_weights = pop_weights_mat[0, :]
acc, predictions = ANN.predict_outputs(best_weights, data_inputs, data_outputs, activation = "sigmoid")
print("Accuracy of the best solution is: ", acc)

plt.plot(accuracies, linewidth = 5, color = "black")
plt.xlabel("Iteration ", fontsize = 20)
plt.ylabel("Fitnes ", fontsize = 20)
plt.xticks(np.arange(0, num_generations + 1, 100), fontsize = 15)
plt.yticks(np.arange(0, 101, 5), fontsize = 15)

f = open("weights_" + str(num_generations) + "_iterations_" + str(mutation_percent) + "%_mutation.pkl", "wb")
pickle.dump(pop_weights_mat, f)
f.close()

Generation:  0


ValueError: cannot reshape array of size 24300 into shape (150,60)